<a href="https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/classification_for_image_tagging/flowers/classification_display_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Display & export results from trained classification models for images
---
*Last Updated 11 Aug 2020*  
*--Update as of 14 Oct 2021--Flower classification pipelines are no longer being updated. As a result, this script is left in its state from 11 Aug 2020. Functions may become deprecated or lose functionality. For updated classification of Flowers/Fruits, [go here](https://github.com/aubricot/computer_vision_with_eol_images/tree/master/classification_for_image_tagging/flower_fruit)--*    

Display classification results from trained classification models on images and verify that they are as expected (or to further fine tune the classification model accordingly, ex: adjust hyperparameters from drop-down menus and re-train). Export resulting classifications to file for use as EOLv3 image tags.

**Notes**
* Change filepaths or information using the form fields to the right of code blocks (also noted in code with 'TO DO')
* Make sure to set the runtime to GPU Hardware Accelerator with a High Ram Runtime Shape (Runtime -> Change runtime type) 

## Imports   
---

In [ ]:
# Mount google drive to import/export files
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# For working with data and reading/displaying images
import itertools
import os
import numpy as np
import pandas as pd
from skimage.transform import resize
import matplotlib.pyplot as plt
import time

# For image classification and training
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

## Run images from URL through trained classifer without exporting results
---

### Define functions and select model

In [ ]:
import time
# Load trained model from path
start = time.time()
model_selection = ("06_inception", 224) #@param ["(\"20_fromscratch\", 150)", "(\"02_mobilenetssd\", 224)", "(\"06_inception\", 224)"] {type:"raw"}
TRAIN_SESS, pixels = model_selection
saved_model_path = '/content/drive/My Drive/summer20/classification/flowers/saved_models/' + TRAIN_SESS
flower_model = tf.keras.models.load_model(saved_model_path)
end = time.time()
print("Build time: {} sec".format(format(end-start, '.3f')))
dataset_labels = ['Branch', 'Entire', 'Flower', 'Fruit', 'Leaf', 'Stem']

# Load in image from URL
# Modified from https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/saved_model.ipynb#scrollTo=JhVecdzJTsKE
def image_from_url(url, fn):
  file = tf.keras.utils.get_file(fn, url) # Filename doesn't matter
  disp_img = tf.keras.preprocessing.image.load_img(file)
  img = tf.keras.preprocessing.image.load_img(file, target_size=[pixels, pixels])
  x = tf.keras.preprocessing.image.img_to_array(img)
  #x = tf.keras.applications.mobilenet_v2.preprocess_input(
    #x[tf.newaxis,...])
  x = tf.keras.applications.inception_v3.preprocess_input(
    x[tf.newaxis,...])
  return x, disp_img

# Run image through classifier
def run_model(x):
  infer = flower_model.signatures["serving_default"]
  label_id = infer(tf.constant(x))[flower_model.output_names[0]]
  label = dataset_labels[np.argmax(label_id)]
  confidence = format(np.amax(label_id)*100, '.3f')
  return label, confidence

### Single Image

In [ ]:
# TO DO: Insert image URL using form field to the right
url = "https://content.eol.org/data/media/80/ce/d7/542.6789991664.jpg" #@param {type:"string"}
fn = "angiosperm_image.jpg"
img, disp_img = image_from_url(url, fn)
label, confidence = run_model(img)

# Plot and show cropping boxes on images
_, ax = plt.subplots(figsize=(10, 10))
ax.imshow(disp_img)
plt.axis('off')
plt.title("Prediction: {}, Confidence: {}%".format(label, confidence))

### Multiple images

In [ ]:
# TO DO: Enter URLs here
url1 = 'https://content.eol.org/data/media/80/ce/d7/542.6789991664.jpg' #@param {type:"string"}
url2 = 'https://content.eol.org/data/media/80/d7/17/542.6872776289.jpg' #@param {type:"string"}
url3 = 'https://content.eol.org/data/media/81/12/42/542.7670415710.260x190.jpg' #@param {type:"string"}
urls = [url1, url2, url3]
print(urls)

# Classify images from URL and display results
for im_num, url in enumerate(urls, start=1):
  fn = str(im_num) + '.jpg'
  img, disp_img = image_from_url(url, fn)
  # Record inference time
  start_time = time.time()
  # Classify image
  label, confidence = run_model(img)
  end_time = time.time()
  # Display progress message after each image
  print('Classification complete in {} of {} images'.format(im_num, len(urls)))

  # Plot and show cropping boxes on images
  _, ax = plt.subplots(figsize=(10, 10))
  ax.imshow(disp_img)
  plt.axis('off')
  plt.title("{}) Prediction: {}, Confidence: {}%, Inference Time: {}".format(im_num, label, confidence, format(end_time-start_time, '.2f')))

### Run images from EOL image bundles

In [ ]:
# Read in EOL image bundle dataframe
# TO DO: Type in image bundle address using form field to right
bundle = 'https://editors.eol.org/other_files/bundle_images/files/images_for_Angiosperms_20K_breakdown_000031.txt' #@param {type:"string"}
df = pd.read_csv(bundle, sep='\t', header=0)
df.head()

In [ ]:
# Optional: Run inference for taxon-specific images only
# TO DO: Type in the taxon you'd like to inspect results for using form field to right
taxon = "" #@param {type:"string"}
df = df.loc[df.ancestry.str.contains(taxon, case=False, na=False)]
print(df.head())

In [ ]:
# Set number of seconds to timeout if image url taking too long to open
import socket
socket.setdefaulttimeout(10)

# TO DO: Set start and end rows to run inference for from EOL image bundle using form field to right
start =  815#@param {type:"integer"}
end =  830#@param {type:"integer"}

# Loop through EOL image bundle to classify images and generate tags
for i, row in df.iloc[start:end].iterrows():
  try:
    # Get url from image bundle
    url = df['eolMediaURL'][i]
    # Read in image from url
    fn = str(i) + '.jpg'
    img, disp_img = image_from_url(url, fn)
    # Record inference time
    start_time = time.time()
    # Run inference/Classify image
    label, confidence = run_model(img)
    end_time = time.time()
    # Display progress message after each image
    print('Classification complete in {} of {} images'.format(i, len(df)))

    # Show classification results for images
    _, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(disp_img)
    plt.axis('off')
    plt.title("{}) Prediction: {}, Confidence: {}%, Inference Time: {}".format(i, label, confidence, format(end_time-start_time, '.3f')))

    # Export tagging results to tsv
    # Define variables for export
    identifier = df['identifier'][i]
    dataObjectVersionID = df['dataObjectVersionID'][i]
    ancestry = df['ancestry'][i]

  except:
    print('Check if URL from {} is valid'.format(url))

## Run images from URL through trained classifer & export tagging results to tsv
---

### Define functions and select model

In [ ]:
import csv

# Load trained model from path
model_selection = ("02_mobilenetssd", 224) #@param ["(\"20_fromscratch\", 150)", "(\"02_mobilenetssd\", 224)", "(\"06_inception\", 224)"] {type:"raw"}
TRAIN_SESS, pixels = model_selection
saved_model_path = '/content/drive/My Drive/summer20/classification/saved_models/' + TRAIN_SESS
flower_model = tf.keras.models.load_model(saved_model_path)
dataset_labels = ['Branch', 'Entire', 'Flower', 'Fruit', 'Leaf', 'Stem']

# Write header row of output crops file
# TO DO: Change file name for each bundle/run abcd if doing 4 batches using form field to right
tags_file = 'angiosperm_tags_20k_d' #@param {type:"string"}
tags_fpath = '/content/drive/My Drive/summer20/classification/results/' + tags_file + '.tsv'
with open(tags_fpath, 'a') as out_file:
                  tsv_writer = csv.writer(out_file, delimiter='\t')
                  tsv_writer.writerow(["eolMediaURL", "identifier", "dataObjectVersionID", "ancestry", "tag"])

# Load in image from URL
# Modified from https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/saved_model.ipynb#scrollTo=JhVecdzJTsKE
def image_from_url(url, fn):
  file = tf.keras.utils.get_file(fn, url) # Filename doesn't matter
  disp_img = tf.keras.preprocessing.image.load_img(file)
  img = tf.keras.preprocessing.image.load_img(file, target_size=[pixels, pixels])
  x = tf.keras.preprocessing.image.img_to_array(img)
  x = tf.keras.applications.inception_v3.preprocess_input(
    x[tf.newaxis,...]) # tried using mobilenet_v2 for mobilenet model and made no difference, keep same for all
  return x, disp_img

# Run image through classifier
def run_model(x):
  infer = flower_model.signatures["serving_default"]
  label_id = infer(tf.constant(x))[flower_model.output_names[0]]
  label = dataset_labels[np.argmax(label_id)]
  confidence = format(np.amax(label_id)*100, '.2f')
  return label, confidence

### Run images from EOL image bundles

In [ ]:
# Read in EOL image bundle dataframe
# TO DO: Type in image bundle address using form field to right
bundle = 'https://editors.eol.org/other_files/bundle_images/files/images_for_Angiosperms_20K_breakdown_000031.txt' #@param {type:"string"}
df = pd.read_csv(bundle, sep='\t', header=0)
df.head()

In [ ]:
# Set number of seconds to timeout if image url taking too long to open
import socket
socket.setdefaulttimeout(10)

# TO DO: Set start and end rows to run inference for from EOL image bundle using form field to right
start =  19136#@param {type:"integer"}
end = 20000 #@param {type:"integer"}

# Loop through EOL image bundle to classify images and generate tags
for i, row in df.iloc[start:end].iterrows():
  try:
    # Get url from image bundle
    url = df['eolMediaURL'][i]
    # Read in image from url
    fn = str(i) + '.jpg'
    img, disp_img = image_from_url(url, fn)
    # Record inference time
    start_time = time.time()
    # Run inference/Classify image
    label, confidence = run_model(img)
    end_time = time.time()
    # Display progress message after each image
    print('Classification complete in {} of {} images'.format(i, len(df)))

    # Show classification results for images
    #_, ax = plt.subplots(figsize=(10, 10))
    #ax.imshow(disp_img)
    #plt.axis('off')
    #plt.title("{}) Prediction: {}, Confidence: {}%, Inference Time: {}".format(i, label, confidence, format(end_time-start_time, '.3f')))

    # Export tagging results to tsv
    # Define variables for export
    if label=="Flower":
        identifier = df['identifier'][i]
        dataObjectVersionID = df['dataObjectVersionID'][i]
        ancestry = df['ancestry'][i]
        with open(tags_fpath, 'a') as out_file:
          tsv_writer = csv.writer(out_file, delimiter='\t')
          tsv_writer.writerow([url, identifier, dataObjectVersionID, ancestry, label])

  except:
    print('Check if URL from {} is valid'.format(url))

## Display exported tagging results on EOL images
---   
Verify that results on EOL image bundles are as expected by randomly sampling images from tagging files

In [ ]:
# Read in EOL image tagging dataset
# TO DO: Type in image tag filepath in form field to right
tags_file = 'angiosperm_tags_20k_a' #@param {type:"string"}
tags_fpath = '/content/drive/My Drive/summer20/classification/results/' + tags_file + '.tsv'
df = pd.read_csv(tags_fpath, sep='\t', header=0)
print(df.head())

# Function to load in image from URL
# Modified from https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/saved_model.ipynb#scrollTo=JhVecdzJTsKE
def image_from_url(url, fn):
  file = tf.keras.utils.get_file(fn, url) # Filename doesn't matter
  disp_img = tf.keras.preprocessing.image.load_img(file)
  img = tf.keras.preprocessing.image.load_img(file, target_size=[224, 224])
  x = tf.keras.preprocessing.image.img_to_array(img)
  x = tf.keras.applications.inception_v3.preprocess_input(
    x[tf.newaxis,...]) # tried using mobilenet_v2 for mobilenet model and made no difference, keep same for all
  return x, disp_img

In [ ]:
# Optional: Display tags for taxon-specific images only
# TO DO: Type in the taxon you'd like to inspect results for using form field to right
taxon = "" #@param {type:"string"}
df = df.loc[df.ancestry.str.contains(taxon, case=False, na=False)]
print(df.head())

In [ ]:
import socket
import random
socket.setdefaulttimeout(10)

# TO DO: Set n below to randomly select n images from EOL image bundle tags using form field to right
start =  0
end = len(df)
n = 50 #@param {type:"slider", min:0, max:50, step:10}
rand = random.sample(range(start, end), n)

# Loop through EOL tag dataset to display and evaluate results on images
for i, row in df.iloc[rand].iterrows():
  try:
    # Get url from tag dataset
    url = df['eolMediaURL'][i]
    # Read in image from url
    fn = str(i) + '.jpg'
    img, disp_img = image_from_url(url, fn)
    # Get tag from dataset
    label = df['tag'][i]
    # Display progress message after each image
    print('{} of {} images loaded'.format(i, len(df)))

    # Show classification results for images
    _, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(disp_img)
    plt.axis('off')
    plt.title("{}) Prediction: {}".format(i, label))
  
  except:
    print('Check if URL from {} is valid'.format(url))